# Chapter 16. 스파크 어플리케이션 개발하기

## Writing Spark Application
- 스파크 어플리케이션: 스파크 클러스터와 코드의 조합
- standalone에서: 클러스터는 로컬 모드, 애플리케이션은 미리 정의된 모드

### A Simple Scala-Based App
- Scala는 Spark의 Native언어. 그냥 스칼라 어플리케이션 만드는 거랑 같음
- TIP: 스칼라는 좀 빡칠 수 있지만, 네 출신에 따라 배울 가치가 있을 수도? 꼭 다 배워야 하는 것도 아님
- JVM 기반 빌드 도구인 sbt나 Maven으로 애플 빌드 가능
- sbt가 가장 쉬움
- scala가 당장은 관심이 없고, 어차피 자세한 건 여기서도 다루지 않아(beyond the scope) 생략

### Writing Python Applications
- 파이스파크 짜는 건 걍 파이썬 코드 짜는 거랑 다를 것 없음
- 빌드 없이 스크립트만 있으면 됨
- 코드 재사용을 위해 egg나 ZIP파일로 파이썬 파일을 마는 것이 일반적
  - spark-submit의 --py-files 인수 사용
- 코드를 돌릴 때는, Scala/Java main class에 상응하는 SparkSession을 만들어 실행 스크립트로 지정한다

```
# in Python
  from __future__ import print_function
  if __name__ == '__main__':
      from pyspark.sql import SparkSession
      spark = SparkSession.builder \
          .master("local") \
          .appName("Word Count") \
          .config("spark.some.config.option", "some-value") \
          .getOrCreate()
      print(spark.range(5000).where("id > 500").selectExpr("sum(id)").collect())
```
 
- 위의 코드로 만들어진 Sparksession을 통해 app과 소통. 단, 런타임에만 전달
- pip을 이용해 pyspark의 종속성을 지정(pip으로 설치하란 의미인듯)

**Running the application**
- 아래 코드 예시처럼 spark-submit하면 됨

```$SPARK_HOME/bin/spark-submit --master local pyspark_template/main.py```

### Writing Java Applications
- 자바는 스칼라에서와 같은데, 종속성만 다름. 자세한 건 생략
- 자바 몰라요 죄송 ....

## Testing Spark Applications
- 어케 스파크 코드를 짜는지는 알았으니 이제 테스트를 보자

### Strategic Principles
- 데이터 파이프라인과 Spark App을 테스트하는 건 개중요

**Input data resilience**
- 다양한 종류의 입력에 탄력적으로 대응하도록 데이터 파이프라인을 짜야함
- 비즈니스가 바뀌면 데이터도 바뀌므로, 어느 정도 변경에 대해서는 탄력적이어야 함
- 아님 발생하는 장애에 우아하고 탄력적으로 대응할 수 있던가
- 엣지 케이스 잘 커버해야하고, 알람은 알맞은 때에만 울리도록 하자

**Business logic resilience and evolution**
- 파이프라인의 비즈니스 논리도 변경될 가능성이 높음
- 더 중요한 것 원천 데이터에서 추론된 게 개발자가 의도한 것대로 된 것이냐 -> 실제 데이터에서 로버스트한 논리 검사를 해야함
- 기능적인 동작을 확인하기 위해서만 스파크 unit test를 하는 게 아니라, 비즈니스 파이프라인이 잘 동작하는지 확인해야함

**Resilience in output and atomicity**
- 입력, 로직을 확인했으면 이제 결과를 봐야함. 일단 출력 스키마를 확인해보자
- 데이터를 쓰고, 다시 읽지 않은 상태에서 끝나는 건 드문 일. 
  - 즉 파이프라인의 output은 보통 다른 파이프라인의 input
  - 따라서 다음 파이프라인의 consumer가 데이터의 "state"를 이해해야함
  - 잘 완료됐는지, 업데이트 빈도는 어떻게 되는지
- 사실 이건 스파크랑 별개로 데이터 파이프라인을 구축할 때 일반적인 원칙임

### Tactical Takeaways
- 위에서는 개념적인 걸(전략적 사고)를 봤으니, 여기서는 구체적인 전략(전술)을 보겠음
- 가장 좋은 접근 방식은 
  - 적절한 unit test롤 비즈니스 로직을 확인하고
  - 입력 데이터 변경에 탄력성을 확보하거나 스키마를 고정시키는 것
- 어떻게 하는지 결정은 개발자에게 달려있음. 비즈니스 도메인이나 관련 전문성에 따라 달라질 수 있기 때문

**Managing SparkSessions**
- local모드에서 JUnit이나 ScalaTest를 돌리면 unit test는 쉽게 할 수 있음
- SparkSession은 한 번만 생성한 뒤, 생성된 session을 관련 함수/클래스에 전달하는 것이 좋다

**Which Spark API to Use?**
- RDD, DF, Dataset 등 API가 많은데, 정답은 없음
- 단 API는 뭘 쓰든 각 기능의 입/출력 유형을 문서화하고 테스트하는 게 좋음
- Type-safe API??
  - The type-safe API automatically enforces a minimal contract for your function that makes it easy for other code to build on it
- DF나 SQL 같은 동적 프로그래밍의 경우 무슨 일이 날 줄 모르기 때문에 input에 대한 유형화/테스트에 시간이 쓰임
- RDD API처럼 정적이면 꼭 기능이 필요한 경우에만 써야 함. 최적화도 안되고 기능도 제한적이라서.
- 일반적으로 대규모 App이나 성능 이슈가 있으면 Scala나 Java같은 정적 언어가 낫지만, 그렇지 않으면 Python이나 R 쓰자.
- 단, 스파크 코드는 모든 언어의 표준 유닛 테스트 프레임 워크에서 테스트 가능해야함

### Connecting to Unit Testing Frameworks
- unit test를 할 땐 해당 언어에 맞는 표준 프레임 워크(UNist이나 ScalaTest)를 써라
- 각 테스트에 대한 스파크 세션을 만들고 정리하도록 테스트 하니스test harness를 써라(?)
- 프레임 워크마다 다르기 때문에, 템플릿에 일부 유닛 테스크 코드를 포함함

### Connecting to Data Sources
- 가능하면 테스트 코드가 프로덕션 데이터 소스에 연결되지 않아야 함 -> 테스트 코드를 쉽게 분리하여 실행
- 비즈니스 로직 기능을, df나 dataset을 입력 받도록 하면 됨
- Structured API를 쓸 때도 dummy dataset을 table르 등록하고 이용

## The Development Process
- 보통 notebook같은 interactive한 곳에서 코딩하고, 나중에 library나 package로 이동
- 당장 돌려보기 편리하니까. databricks같은 것도 좋음
- 로컬 컴퓨터에서 쓸 때는 spark-shell이 좋음
- 그 중에서 spark-submit은 product app을 위한 것

## Launching Applications
- 스파크 app을 돌리는 가장 일반적인 방법은 spark-submit
- 옵션과 JAR 등을 다음과 같이 지정하면 됨

```
  ./bin/spark-submit \
    --class <main-class> \
    --master <master-url> \
    --deploy-mode <deploy-mode> \
    --conf <key>=<value> \
    ... # other options
    <application-jar-or-script> \
    [application-arguments]
```

- 클러스터 모드로 할지 클라이언트 모드로 할 지도 지정할 수 있음
- 근데 executor와 driver 사이의 지연을 줄이기 위해서는 클러스터 모드를 사용하는 게 좋음

### Application Launch Examples
- 아래 코드에서는 옵션들을 살펴보면 좋겠음

```
 ./bin/spark-submit \
    --class org.apache.spark.examples.SparkPi \
    --master spark://207.184.161.138:7077 \
    --executor-memory 20G \
    --total-executor-cores 100 \
    replace/with/path/to/examples.jar \
    1000
```

- 아래 스니펫은 파이썬도 동일

```
  ./bin/spark-submit \
    --master spark://207.184.161.138:7077 \
    examples/src/main/python/pi.py \
    1000
```

- local로 하면 로컬 모드이고, local[*]로 하면 모든 코어 사용

## Configuring Applications
- 스파크에는 다양한 구성이 있고, 대부분은 아래에 속함
  - Application properties Runtime environment
  - Shuffle behavior
  - Spark UI
  - Compression and serialization Memory management Execution behavior Networking
  - Scheduling
  - Dynamic allocation Security Encryption
  - Spark SQL
  - Spark streaming
  - SparkR
- 스파크에서 configure는 다음의 세 부분에서
  - SparkConf가 대부분의 파라미터 제어
  - Java 시스템 속성
  - 하드코딩된 config 파일
- 스파크의 root에서 conf아래에 템플릿들이 있음
  - 하드코딩하거나 런타임에서 지정하여 템플릿 사용가능
  - 환경변수를 이용해서 설정할 수도 있음
  - log4j.properties로 로깅 구성 가능
    - 이거 해킹 이슈 있지 않았음?
    
    
### The SparkConf
- 얘가 모든 App 구성 관리함
- 아래 코드 참조. 참고로 SparkConf는 immutable

```
# in Python
  from pyspark import SparkConf
  conf = SparkConf().setMaster("local[2]").setAppName("DefinitiveGuide")\
    .set("some.conf", "to.some.value")
```

### Application Properties
- App 속성은 spark-submit이나 Spark App을 만들 때 설정하는 것. 표 16-3 참조
- 드라이버의 4040 포트, "Environment" 탭에서 속성 확인 가능
- 명시적으로 지정된 값 외에는 모두 디폴트

### Runtime Properties
- 일반적이지는 않지만 런타임 환경을 config할 때도 있음
- 여백이 부족해 다 적지는 않음. 스파크 설명서 참조
- driver와 execute, python worker config, 기타 logging 등 설정 가능

### Execution Properties
- 실제 실행을 세부적으로 제어하기 땜에 가장 관련있는 것. 
- 여백이 없어 생략. 도큐 참조.
- spark.exeutor.core와 같은 것 변경 가능

### Configuring Memory Management
- 최적화를 위해 메모리 옵션을 수동 관리해야할 때가 있ㅇ므
- 2.X에서는 자동 메모리 관리로 없어진 게 많아서 end user랑은 상관 ㅇ벗는 게 많음
- 여백 없어 생략했으니 도큐 참조

### Configuring Shuffle Behavior
- 오버헤드 문제로 shuffle이 병목이 될 수 있음
- low level로 제어할 수 있는 게 좀 있음
- 여백 없어 생략했으니 도큐 참조

### Environmental Variables
- 환경 변수로 특정 스파크 설정 구성 가능
- 설치된 디렉토리의 conf/spark-env.sh 스크립트에서 읽어옴
- Standalone이나 Mesos에서, hostname과 같은 기기 관련 정보를 제공할 수 있음?
- spark-env.sh는 기본적으로는 존재하지 않는 파일 -> conf/spark-env.sh.template을 복사해서 사용

### Job Scheduling Within an Application
- 여러 스레드?에서 동시에 작업이 submit되면 동시에 병렬로 job이 실행될 수 있음
- 스파크 스케쥴러는 thread-safe?이고 동시에 여러 요청을 처리할 수 있게 해줌
- 기본적으로는 FIFO인데, 맨 앞의 작업이 좀 작아 여유가 되면 뒤의 작업을 동시에 처리하기도 ㅎ마
- 공평하게 자원을 배분할 수도 있음 -> 라운드 로빈 방식
  - spark.scheduler.mode를 FAIR로 설정
  - job들을 pool로 그룹화하고 각 풀에 대해 서로 다른 예약옵션/가중치 설정을 지원 -> 우선순위 풀 생성 가능
  - 새로 submit된 job은 default pool로 옮기는데, 따로 pool을 생성해줄 수도 있음
  - 스레드가 연결된 풀을 지우려면 null로 셋업